In [4]:
%matplotlib inline
import numpy as np
from scipy.special import sph_harm

import matplotlib.pyplot as plt

## Functions

In [16]:
def x_lm(l, m, theta, phi):
    """Vector spherical harmonics as defined in Hill (1954)."""
    # m, l are integers and abs(m) <= l
    if ( int(m) != m ) or ( int(l) != l ):
        return np.nan, np.nan
    elif np.abs(m) > l:
        return np.nan, np.nan
    
    # Be careful that scipy defines polar and azimuthal
    # angles in an opposite convention
    # Also (-1)^m typically in Y_lm is in the associated Legendre function
    theta_comp = -1 * m * sph_harm(m, l, phi, theta) / ( np.sqrt(l * (l+1)) * np.sin(theta) )
    
    # Derivative of Y_ml w.r.t theta
    # https://functions.wolfram.com/Polynomials/SphericalHarmonicY/20/ShowAll.html
    if (m == l):
        dydtheta = m * 1/(np.tan(theta)) * sph_harm(m, l, phi, theta)
    else:
        dydtheta = m * 1/(np.tan(theta)) * sph_harm(m, l, phi, theta) + \
                   np.sqrt((l-m) * (l+m+1)) * np.exp(-1j*phi) * sph_harm(m+1, l, theta, phi)
    phi_comp = ( -1j / np.sqrt(l*(l+1)) ) * dydtheta
    
    # X_ml has no r-component
    return np.asarray([theta_comp, phi_comp])

In [18]:
# Unit vectors in spherical coords
def e_phi(theta, phi):
    pass

def e_theta(theta, phi):
    pass

# Polarization vectors
def e1(theta, phi):
    return 1j * e_phi(theta, phi)

def e2(theta, phi):
    return e_theta(theta, phi)

# Vectors used for `C_p_lm` below
def u_p(p, theta, phi):
    if p == 'TE':
        return e_theta(theta, phi) + 1j * e_phi(theta, phi)
    elif p == 'TM':
        return e_phi(theta, phi) - 1j * e_theta(theta, phi)

In [23]:
def cbar_p_lmg(l, m, g, p, theta, phi):
    """Eq. (A2), (A3)"""
    if p == 'TE':
        pass
    elif p == 'TM':
        pass

In [17]:
def z_lm(l, m, na, f, w):
    """Z_lm function related to Gaussian beam defined in Eq. (B10)
    
    Params
      l, m: integer indices
      na  : trapping NA 
      f   : focal length
      w   : waist
    """
    if ( int(m) != m ) or ( int(l) != l ):
        return np.nan, np.nan
    elif np.abs(m) > l:
        return np.nan, np.nan
    
    theta_max = np.arcsin(na)
    
    tt = np.linspace(0, theta_max, 200)
    int_factor = np.sin(tt) * np.sqrt(np.abs(np.cos(tt))) * np.exp(-1*f*f*np.sin(tt)*np.sin(tt) / (w*w))
    x_theta, x_phi = x_ml(m, l, tt, 0)
    
    # Integrate over theta
    theta_comp = np.trapz(x_theta*int_factor, tt)
    phi_comp   = np.trapz(x_phi  *int_factor, tt)
    
    return np.asarray([theta_comp, phi_comp])

In [20]:
def C_p_lm(l, m, p, na, f, w):
    """C_p_lm function defined in Eq. (B9)"""
    if p == 'TE':
        pass
    elif p == 'TM':
        pass

In [22]:
def Ax_p_lmg():
    """Eq. (A20)"""
    pass

def Ay_p_lmg():
    """Eq. (A21)"""
    pass

def Az_p_lmg():
    """Eq. (A22)"""
    pass

In [21]:
def R_p_l():
    if p == 'TE':
        pass
    elif p == 'TM':
        pass

def S_p_l():
    if p == 'TE':
        pass
    elif p == 'TM':
        pass
    
def psi_p_l():
    if p == 'TE':
        pass
    elif p == 'TM':
        pass

## Focused Gaussian beam